In [1]:
from sklearn.datasets import load_iris

iris = load_iris()

print(iris.DESCR)

.. _iris_dataset:

Iris plants dataset
--------------------

**Data Set Characteristics:**

:Number of Instances: 150 (50 in each of three classes)
:Number of Attributes: 4 numeric, predictive attributes and the class
:Attribute Information:
    - sepal length in cm
    - sepal width in cm
    - petal length in cm
    - petal width in cm
    - class:
            - Iris-Setosa
            - Iris-Versicolour
            - Iris-Virginica

:Summary Statistics:

============== ==== ==== ======= ===== ====================
                Min  Max   Mean    SD   Class Correlation
============== ==== ==== ======= ===== ====================
sepal length:   4.3  7.9   5.84   0.83    0.7826
sepal width:    2.0  4.4   3.05   0.43   -0.4194
petal length:   1.0  6.9   3.76   1.76    0.9490  (high!)
petal width:    0.1  2.5   1.20   0.76    0.9565  (high!)
============== ==== ==== ======= ===== ====================

:Missing Attribute Values: None
:Class Distribution: 33.3% for each of 3 classes.
:Cr

In [2]:
import pandas as pd

# Cria uma tabela (DataFrame) com os dados e nomes das colunas
df = pd.DataFrame(data=iris.data, columns=iris.feature_names)

# Adiciona uma coluna para dizer a espécie de cada flor
df['species'] = iris.target_names[iris.target]

# O comando .head() mostra as 5 primeiras linhas da nossa tabela
df.head()

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),species
0,5.1,3.5,1.4,0.2,setosa
1,4.9,3.0,1.4,0.2,setosa
2,4.7,3.2,1.3,0.2,setosa
3,4.6,3.1,1.5,0.2,setosa
4,5.0,3.6,1.4,0.2,setosa


In [3]:
from sklearn.model_selection import train_test_split

# Nossas 'features' (características) são as 4 colunas com medidas
X = df[['sepal length (cm)', 'sepal width (cm)', 'petal length (cm)', 'petal width (cm)']]

# Nosso 'target' (alvo) é a coluna 'species'
# Para este modelo simples, vamos começar com um desafio mais fácil:
# A IA vai dizer apenas se uma flor é "setosa" (Sim/Não).
# O número '0' representa 'setosa', e '1' representa 'não-setosa'.
y = (df['species'] != 'setosa').astype(int)


# Agora, dividimos os dados: 80% para treino, 20% para teste.
# O random_state=42 garante que a divisão seja sempre a mesma, para podermos repetir os resultados.
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Vamos ver quantas amostras ficaram em cada conjunto
print("Tamanho do conjunto de treino:", len(X_train))
print("Tamanho do conjunto de teste:", len(X_test))

Tamanho do conjunto de treino: 120
Tamanho do conjunto de teste: 30


In [4]:
import numpy as np

# A função Sigmoid que transforma um número em probabilidade
def sigmoid(z):
    return 1 / (1 + np.exp(-z))

# A função de predição do nosso modelo
def predict(features, weights, bias):
    z = np.dot(features, weights) + bias
    return sigmoid(z)
    

In [5]:
# 1. Inicialização
# Começamos com pesos e bias aleatórios (ou zerados).
# Temos 4 features, então precisamos de 4 pesos.
weights = np.zeros(X_train.shape[1])
bias = 0

# 2. Hiperparâmetros
# São as "configurações" do nosso processo de aprendizado.
learning_rate = 0.01  # O "tamanho do passo" que a IA dá a cada ajuste.
epochs = 1000         # O número de vezes que a IA vai "olhar" para o conjunto de dados completo.

# 3. O Loop de Treinamento
for epoch in range(epochs):
    # Fazer previsões para todos os dados de treino
    predictions = predict(X_train, weights, bias)

    # Calcular o erro
    # A diferença entre a previsão e a resposta real (y_train).
    error = predictions - y_train

    # Calcular os gradientes (a "direção" do erro)
    # Isso vem de uma derivada matemática, mas a ideia é ver como cada peso contribuiu para o erro.
    gradient_weights = np.dot(X_train.T, error) / len(y_train)
    gradient_bias = np.sum(error) / len(y_train)

    # Atualizar os pesos e o bias para diminuir o erro
    # Damos um pequeno passo na direção oposta ao gradiente.
    weights = weights - learning_rate * gradient_weights
    bias = bias - learning_rate * gradient_bias

    # A cada 100 épocas, vamos imprimir o progresso (opcional)
    if (epoch + 1) % 100 == 0:
        # Calculamos o erro médio (Função de Custo) para ver se está diminuindo
        cost = -np.mean(y_train * np.log(predictions) + (1 - y_train) * np.log(1 - predictions))
        print(f"Época {epoch + 1}/{epochs}, Custo: {cost:.4f}")


print("\n--- Treinamento Concluído! ---")
print("Pesos aprendidos:", weights)
print("Bias aprendido:", bias)


Época 100/1000, Custo: 0.3270
Época 200/1000, Custo: 0.2179
Época 300/1000, Custo: 0.1619
Época 400/1000, Custo: 0.1286
Época 500/1000, Custo: 0.1067
Época 600/1000, Custo: 0.0913
Época 700/1000, Custo: 0.0799
Época 800/1000, Custo: 0.0710
Época 900/1000, Custo: 0.0640
Época 1000/1000, Custo: 0.0583

--- Treinamento Concluído! ---
Pesos aprendidos: [-0.25465902 -0.92962653  1.47018452  0.65708141]
Bias aprendido: -0.17047600334737856


In [6]:
# 1. Fazer previsões no conjunto de TESTE
# Usamos os pesos e o bias que acabamos de aprender.
predictions_test = predict(X_test, weights, bias)

# As previsões são probabilidades (ex: 0.01, 0.98, etc.)
# Vamos converter para uma decisão final:
# Se a probabilidade for > 0.5, a previsão é 1 (não-setosa)
# Se for <= 0.5, a previsão é 0 (setosa)
final_predictions = (predictions_test > 0.5).astype(int)

# 2. Comparar as previsões com as respostas reais
print("Previsões da IA:", final_predictions)
print("Respostas Reais:", y_test.values)

Previsões da IA: [1 0 1 1 1 0 1 1 1 1 1 0 0 0 0 1 1 1 1 1 0 1 0 1 1 1 1 1 0 0]
Respostas Reais: [1 0 1 1 1 0 1 1 1 1 1 0 0 0 0 1 1 1 1 1 0 1 0 1 1 1 1 1 0 0]


In [7]:
# Compara onde as previsões foram iguais às respostas reais
acertos = (final_predictions == y_test.values)

# Calcula a média de acertos (True=1, False=0) e multiplica por 100
acuracia = np.mean(acertos) * 100

print(f"\n--- Resultado Final ---")
print(f"A Acurácia da nossa Mini IA no conjunto de teste é: {acuracia:.2f}%")



--- Resultado Final ---
A Acurácia da nossa Mini IA no conjunto de teste é: 100.00%


In [8]:
# Importa a biblioteca para exibir dados de forma mais bonita
from IPython.display import display

print("--- Testando Flores Individuais do Conjunto de Teste ---")
print("Previsão da IA (0 = Setosa, 1 = Não-Setosa) vs Real\n")

# Vamos pegar as 5 primeiras flores do conjunto de teste para analisar
num_flores_para_testar = 5

for i in range(num_flores_para_testar):
    # Pega as características (features) de uma única flor do conjunto de teste
    # Usamos .iloc[i] para selecionar a i-ésima linha e .values para pegar os números
    flor_individual_features = X_test.iloc[i].values.reshape(1, -1) # reshape(1, -1) é para o predict funcionar com uma única flor

    # Pega a resposta real para essa flor
    flor_individual_real_species = y_test.iloc[i]

    # A IA faz a previsão para essa flor
    prob_predita = predict(flor_individual_features, weights, bias)[0] # [0] para pegar o valor único
    predicao_final_ia = int(prob_predita > 0.5)

    # Imprime os resultados
    print(f"Flor {i+1}:")
    print(f"  Características (cm): Comprimento Sépala={flor_individual_features[0][0]:.2f}, Largura Sépala={flor_individual_features[0][1]:.2f}, Comprimento Pétala={flor_individual_features[0][2]:.2f}, Largura Pétala={flor_individual_features[0][3]:.2f}")
    print(f"  Probabilidade de NÃO ser Setosa (IA): {prob_predita:.4f}")
    print(f"  Previsão Final da IA: {predicao_final_ia} ( {'Não-Setosa' if predicao_final_ia == 1 else 'Setosa'} )")
    print(f"  Espécie Real:       {flor_individual_real_species} ( {'Não-Setosa' if flor_individual_real_species == 1 else 'Setosa'} )")
    print(f"  Acertou?: {'SIM' if predicao_final_ia == flor_individual_real_species else 'NÃO'}")
    print("-" * 40)

--- Testando Flores Individuais do Conjunto de Teste ---
Previsão da IA (0 = Setosa, 1 = Não-Setosa) vs Real

Flor 1:
  Características (cm): Comprimento Sépala=6.10, Largura Sépala=2.80, Comprimento Pétala=4.70, Largura Pétala=1.20
  Probabilidade de NÃO ser Setosa (IA): 0.9668
  Previsão Final da IA: 1 ( Não-Setosa )
  Espécie Real:       1 ( Não-Setosa )
  Acertou?: SIM
----------------------------------------
Flor 2:
  Características (cm): Comprimento Sépala=5.70, Largura Sépala=3.80, Comprimento Pétala=1.70, Largura Pétala=0.30
  Probabilidade de NÃO ser Setosa (IA): 0.0788
  Previsão Final da IA: 0 ( Setosa )
  Espécie Real:       0 ( Setosa )
  Acertou?: SIM
----------------------------------------
Flor 3:
  Características (cm): Comprimento Sépala=7.70, Largura Sépala=2.60, Comprimento Pétala=6.90, Largura Pétala=2.30
  Probabilidade de NÃO ser Setosa (IA): 0.9992
  Previsão Final da IA: 1 ( Não-Setosa )
  Espécie Real:       1 ( Não-Setosa )
  Acertou?: SIM
------------------